# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Create city dataframe
city_df = pd.read_csv("../output_data/cities.csv")
city_df.head()

,City,Cloudiness,Country,DateTime,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ushuaia,75,AR,2020-01-27 02:21:31,70,-54.80,-68.30,48.20,5.82
1,Yerbogachen,90,RU,2020-01-27 02:21:32,91,61.28,108.01,2.48,11.90
2,Hobart,75,AU,2020-01-27 02:20:17,42,-42.88,147.33,69.01,12.75
3,Port Alfred,18,ZA,2020-01-27 02:21:34,76,-33.59,26.89,62.01,5.99
4,Yellowknife,90,CA,2020-01-27 02:21:35,84,62.46,-114.35,1.40,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=city_df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=.7)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#drop rows with null values
city_df.dropna()

,City,Cloudiness,Country,DateTime,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ushuaia,75,AR,2020-01-27 02:21:31,70,-54.80,-68.30,48.20,5.82
1,Yerbogachen,90,RU,2020-01-27 02:21:32,91,61.28,108.01,2.48,11.90
2,Hobart,75,AU,2020-01-27 02:20:17,42,-42.88,147.33,69.01,12.75
3,Port Alfred,18,ZA,2020-01-27 02:21:34,76,-33.59,26.89,62.01,5.99
4,Yellowknife,90,CA,2020-01-27 02:21:35,84,62.46,-114.35,1.40,6.93
...,...,...,...,...,...,...,...,...,...
575,Acapulco,20,MX,2020-01-27 02:29:29,74,16.86,-99.89,80.60,8.05
576,Magistralnyy,100,RU,2020-01-27 02:33:32,89,56.17,107.45,15.85,3.04
577,Edd,22,ER,2020-01-27 02:33:34,72,13.93,41.69,69.30,2.57
578,Carmen,5,MX,2020-01-27 02:33:09,83,18.63,-91.83,75.20,1.95


In [7]:
#setting conditions to find the ideal spot
#temp under 80 degrees
ideal_df = city_df.loc[city_df["Max Temp"] < 80]
#temp over 70 degrees
ideal_df = ideal_df.loc[ideal_df["Max Temp"] > 70]
# wind speed under 10 mph
ideal_df = ideal_df.loc[ideal_df["Wind Speed"] < 10]
#cloudiness set to 0%
ideal_df = ideal_df.loc[ideal_df["Cloudiness"] == 0]
#print the 
ideal_df

,City,Cloudiness,Country,DateTime,Humidity,Lat,Lng,Max Temp,Wind Speed
99,Lazaro Cardenas,0,MX,2020-01-27 02:23:28,76,17.96,-102.20,73.27,1.01
143,Chatakonda,0,IN,2020-01-27 02:24:23,53,17.55,80.65,73.47,3.22
155,Inhambane,0,MZ,2020-01-27 02:24:36,81,-23.86,35.38,77.40,9.98
308,Giddalur,0,IN,2020-01-27 02:27:37,63,15.35,78.92,73.26,1.32
367,Iralaya,0,HN,2020-01-27 02:28:45,69,15.00,-83.23,78.82,9.69
395,Quelimane,0,MZ,2020-01-27 02:29:19,94,-17.88,36.89,78.80,3.36
468,Imbituba,0,BR,2020-01-27 02:30:50,88,-28.24,-48.67,77.00,5.82
477,Tema,0,GH,2020-01-27 02:31:01,83,5.67,-0.02,78.80,3.36
532,Dunmore Town,0,BS,2020-01-27 02:32:06,53,25.50,-76.65,71.64,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city"s coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Establish a new dataframe for hotels.
hotel_df = ideal_df
#Add column for Hotel, set value to NaN
hotel_df['Hotel Name'] = np.nan

#Setting serach constraints
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

# for statement to check each lat/lng for the closest hote
hotel_count = 0
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    # set up a parameters dictionary
    params = {
        "location": f"{lat}, {lng}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params)
    hotels = response.json()
    results = hotels['results']
    #if statement to take first hotel name if more than one is present
    if len(results) > 0:
        name = hotels['results'][0]['name']
        hotel_df.iloc[hotel_count, -1] = name
    hotel_count = hotel_count + 1

narrowed_city_df = hotel_df
narrowed_city_df


,City,Cloudiness,Country,DateTime,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
99,Lazaro Cardenas,0,MX,2020-01-27 02:23:28,76,17.96,-102.20,73.27,1.01,Hotel de Casablanca
143,Chatakonda,0,IN,2020-01-27 02:24:23,53,17.55,80.65,73.47,3.22,Hotel Sri Balaji A/c
155,Inhambane,0,MZ,2020-01-27 02:24:36,81,-23.86,35.38,77.40,9.98,Hotel Casa Do Capitao
308,Giddalur,0,IN,2020-01-27 02:27:37,63,15.35,78.92,73.26,1.32,Prashant Lodge
367,Iralaya,0,HN,2020-01-27 02:28:45,69,15.00,-83.23,78.82,9.69,NaN
395,Quelimane,0,MZ,2020-01-27 02:29:19,94,-17.88,36.89,78.80,3.36,Hotel Elite
468,Imbituba,0,BR,2020-01-27 02:30:50,88,-28.24,-48.67,77.00,5.82,Praia Hotel Imbituba
477,Tema,0,GH,2020-01-27 02:31:01,83,5.67,-0.02,78.80,3.36,Hotel Marjorie Y
532,Dunmore Town,0,BS,2020-01-27 02:32:06,53,25.50,-76.65,71.64,4.70,"Valentines Resort & Marina, Harbour Island"


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
figure_layout = {
    "width": "900px",
    "height": "600px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}

# Plot map
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…